In [8]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os
from duckduckgo_search import DDGS


print("✅ All imports successful")

✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Lower temperature for more precise tool usage
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [4]:
from langchain_core.tools import tool

@tool
def weather_tool(city: str) -> str:
    """
    Get the current weather for a city (simulated).
    Use this tool when the user asks about weather conditions.
    
    Args:
        city: Name of the city
        
    Returns:
        Simulated weather information
    """
    fake_weather = {
        "lagos": "Sunny, 30°C",
        "abuja": "Cloudy, 27°C",
        "london": "Rainy, 15°C"
    }
    
    return f"Weather in {city.title()}: {fake_weather.get(city.lower(), 'Clear, 25°C')}"


In [5]:
@tool
def dictionary_tool(word: str) -> str:
    """
    Look up the definition of a word.
    Use this tool when the user asks for word meanings.
    
    Args:
        word: Word to define
        
    Returns:
        Definition of the word
    """
    dictionary = {
        "ephemeral": "Lasting for a very short time.",
        "agent": "An entity that acts on behalf of another.",
        "algorithm": "A step-by-step procedure for calculations."
    }
    
    return dictionary.get(
        word.lower(),
        f"No definition found for '{word}'."
    )



In [9]:
from duckduckgo_search import DDGS

@tool
def web_search(query: str) -> str:
    """
    Search the web for information using DuckDuckGo.
    Use this tool when the user asks to search for recent or unknown information.
    
    Args:
        query: Search query
        
    Returns:
        Top search results
    """
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=3):
            results.append(f"- {r['title']}: {r['body']}")
    
    return "\n".join(results) if results else "No results found."


In [10]:
tools = [weather_tool, dictionary_tool, web_search]
llm_with_tools = llm.bind_tools(tools)

print("✅ Tools bound:", [t.name for t in tools])


✅ Tools bound: ['weather_tool', 'dictionary_tool', 'web_search']


In [11]:
from langchain_core.messages import SystemMessage
from langgraph.graph import MessagesState

sys_msg = SystemMessage(content="""
You are a helpful assistant with access to tools.

Use:
- weather_tool for weather questions
- dictionary_tool for word definitions
- web_search for searching information online

Only use tools when necessary.
""")

def assistant(state: MessagesState) -> dict:
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}


In [12]:
from typing import Literal

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    last_message = state["messages"][-1]
    
    if last_message.tool_calls:
        return "tools"
    return "__end__"


In [13]:
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

agent = builder.compile(checkpointer=MemorySaver())

print("✅ Stateful agent compiled")


✅ Stateful agent compiled


In [ ]:
run_agent("What's the weather in Lagos?")
